# ResNet

梯度消失和梯度下降：

当初始化的网络权值小于1，当层数增多时，小于1的值不断相乘，最后就导致梯度消失的情况出现。同理，当权值过大时，最后大于1的值不断相乘，就会产生梯度爆炸。

网络之所以难以训练，是因为存在着梯度消失的问题，离 loss 函数越远的层，在反向传播的时候，梯度越小，就越难以更新，随着层数的增加，这个现象越严重。之前有两种常见的方案来解决这个问题：

1.按层训练，先训练比较浅的层，然后在不断增加层数，但是这种方法效果不是特别好，而且比较麻烦

2.使用更宽的层，或者增加输出通道，而不加深网络的层数，这种结构往往得到的效果又不好

ResNet 通过引入了跨层链接解决了梯度回传消失的问题。

使用普通的连接，上层的梯度必须要一层一层传回来，而是用残差连接，相当于中间有了一条更短的路，梯度能够从这条更短的路传回来，避免了梯度过小的情况。

假设某层的输入是 x，期望输出是 H(x)， 如果我们直接把输入 x 传到输出作为初始结果，这就是一个更浅层的网络，更容易训练，而这个网络没有学会的部分，我们可以使用更深的网络 F(x) 去训练它，使得训练更加容易，最后希望拟合的结果就是 F(x) = H(x) - x，这就是一个残差的结构

残差网络的结构就是上面这种残差块的堆叠，下面让我们来实现一个 residual block

一如既往的导入库，写训练函数，读取即预处理数据集

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision.datasets import mnist
from torch.utils.data import DataLoader

In [2]:
from datetime import datetime

def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().data
    return num_correct / total


def train(net, train_data, valid_data, num_epochs, optimizer, criterion):
    if torch.cuda.is_available():
        net = net.cuda()
    prev_time = datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0
        train_acc = 0
        net = net.train()
        for im, label in train_data:
            if torch.cuda.is_available():
                im = Variable(im.cuda())  # (bs, 3, h, w)
                label = Variable(label.cuda())  # (bs, h, w)
            else:
                im = Variable(im)
                label = Variable(label)
            # forward
            output = net(im)
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.data
            train_acc += get_acc(output, label)

        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()
            for im, label in valid_data:
                if torch.cuda.is_available():
                    im = Variable(im.cuda())
                    label = Variable(label.cuda())
                else:
                    im = Variable(im)
                    label = Variable(label)
                output = net(im)
                loss = criterion(output, label)
                valid_loss += loss.data
                valid_acc += get_acc(output, label)
            epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), valid_loss / len(valid_data),
                   valid_acc / len(valid_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data),
                          train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)

In [3]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 数据预处理，标准化
    x=np.array([x.tolist()])
    x = torch.from_numpy(x)    
    return x

from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据
train_set = mnist.MNIST('./data', train=True, transform=data_tf,download=True) # 重新载入数据集，申明定义的数据变换
test_set = mnist.MNIST('./data', train=False, transform=data_tf,download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

查看一个batch数据的尺寸，便于网络的设计

In [4]:
x=torch.tensor([])
for i,j in train_data:
#     print(i[0].shape) 
#     print(i[0])
    x=i
    break
print(x.shape)

接下来设计实现一个 residual block

In [5]:
# 利用pytorch的卷积函数，固定kernel_size参数，以定义一个3*3的卷积函数
def conv3x3(in_channel, out_channel, stride=1):
    return nn.Conv2d(in_channel, out_channel, 3, stride=stride, padding=1, bias=False)

尝试用面向对象的写法，之后多采用面向对象的写法！！！

In [6]:
class residual_block(nn.Module):
    def __init__(self, in_channel, out_channel, same_shape=True):
        super(residual_block, self).__init__()
        
#         如果要保持tensor的shape不变，则卷积核的滑动步长改变为2，可达到一个通道数倍增，图像长宽减半的效果
        self.same_shape = same_shape
        stride=1 if self.same_shape else 2
        
        self.conv1 = conv3x3(in_channel, out_channel, stride=stride)
#         每次卷积后进行一次批标准化
        self.bn1 = nn.BatchNorm2d(out_channel)
        
        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel, 1, stride=stride)
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)
        
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(x+out, True)

In [7]:
print("1:",x.shape)
x=nn.Conv2d(1, 32, 3, 1)(x.float())
print("2:",x.shape)

x=nn.MaxPool2d(2, 2)(x)
print("3:",x.shape)

x=residual_block(32,32)(x)
x=residual_block(32,32)(x)
print("4:",x.shape)

x=residual_block(32,64,False)(x)
x=residual_block(64,64)(x)
print("5:",x.shape)


x=residual_block(64,128,False)(x)
x=residual_block(128,128)(x)
print("6:",x.shape)


x=residual_block(128,256,False)(x)
x=residual_block(256,256)(x)
print("7:",x.shape)


x=nn.AvgPool2d(2)(x)
print("8:",x.shape)

x = x.view(x.shape[0], -1)
print("9:",x.shape)

x=nn.Linear(256, 10)(x)
print("10:",x.shape)

ResNet是 residual block 模块的堆叠

In [8]:
class resnet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(resnet, self).__init__()
        self.verbose = verbose
        
        self.block1 = nn.Conv2d(in_channel, 32, 3,1)
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(2, 2),
            residual_block(32, 32),
            residual_block(32, 32)
        )
        
        self.block3 = nn.Sequential(
            residual_block(32, 64, False),
            residual_block(64, 64)
        )
        
        self.block4 = nn.Sequential(
            residual_block(64, 128, False),
            residual_block(128, 128)
        )
        
        self.block5 = nn.Sequential(
            residual_block(128, 256, False),
            residual_block(256, 256),
            nn.AvgPool2d(2)
        )
        
        self.classifier = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x=x.float()
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [9]:
net = resnet(1, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [10]:
train(net, train_data, test_data, 20, optimizer, criterion)

可以看到，训练20次测试集准确率可达0.99555